In [1]:
import pandas as pd

import time
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm

from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os

device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
from torchvision import transforms

import matplotlib.pyplot as plt


In [3]:
device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
class RetinopathyDatasetTrain(Dataset):
    
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
        
        
    def __getitem__(self, idx):
        label = torch.tensor(self.data.loc[idx, "diagnosis"])
        img_name = os.path.join('/kaggle/input/train_images', 
                                self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
       # image = image.resize((256,256), resample=Image.BILINEAR)
        
        if self.transform:
            image = self.transform(image)
        
        sample = {'image': image, "label": label}
            
        return sample
        
        

In [5]:
class RetinopathyDatasetTest(Dataset):
    
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
        
        
    def __getitem__(self, idx):
        img_name = os.path.join('/kaggle/input/test_images', 
                                self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
       # image = image.resize((256,256), resample=Image.BILINEAR)
        
        if self.transform:
            image = self.transform(image)
            
        return {'image': image}

In [6]:
def show_photo(image, label):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.pause(0.001)  # pause a bit so that plots are updated
    print(label)

In [7]:
transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [ ]:
dataset[0]["image"].shape

In [ ]:
print(device)

In [8]:
model = torchvision.models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(2048, 1)

model = model.to(device)

In [9]:
dataset = RetinopathyDatasetTrain(csv_file = "/kaggle/input/train.csv", transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle = True, num_workers = 4)

plist = [
         {'params': model.layer4.parameters(), 'lr': 1e-4, 'weight': 0.001},
         {'params': model.fc.parameters(), 'lr': 1e-3}
         ]

optimizer = optim.Adam(plist, lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)

In [ ]:
since = time.time()
criterion = nn.MSELoss()
num_epochs = 15

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    scheduler.step()
    model.train()
    running_loss = 0.0
    counter = 0
    for _, sample in enumerate(dataloader):
        image = sample["image"]
        label = sample["label"].view(-1,1)
        image = image.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.float)
        optimizer.zero_grad()
        
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * image.size(0)
        counter +=1
        epoch_loss = running_loss / len(dataloader)
        print('Training Loss: {:.4f}'.format(epoch_loss))
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
torch.save(model.state_dict(), "model.bin")
        

Epoch 0/14
----------
Training Loss: 0.1310
Training Loss: 0.2566
Training Loss: 0.4394
Training Loss: 0.5041
Training Loss: 0.6224
Training Loss: 0.7543
Training Loss: 0.9006
Training Loss: 1.0266
Training Loss: 1.1013
Training Loss: 1.2458
Training Loss: 1.4045
Training Loss: 1.4569
Training Loss: 1.5790
Training Loss: 1.6598
Training Loss: 1.7957
Training Loss: 1.9286
Training Loss: 2.0461
Training Loss: 2.0984
Training Loss: 2.1522
Training Loss: 2.3580
Training Loss: 2.4365
Training Loss: 2.5667
Training Loss: 2.6585
Training Loss: 2.7590
Training Loss: 2.8868
Training Loss: 3.0655
Training Loss: 3.1259
Training Loss: 3.2094
Training Loss: 3.3906
Training Loss: 3.4774
Training Loss: 3.6364
Training Loss: 3.7882
Training Loss: 3.9240
Training Loss: 4.0723
Training Loss: 4.1401
Training Loss: 4.2491
Training Loss: 4.3868
Training Loss: 4.5211
Training Loss: 4.6799
Training Loss: 4.7804
Training Loss: 4.8559
Training Loss: 4.9081
Training Loss: 5.0087
Training Loss: 5.1174
Training L

Training Loss: 14.0270
Training Loss: 14.0927
Training Loss: 14.1576
Training Loss: 14.2190
Training Loss: 14.2839
Training Loss: 14.3612
Training Loss: 14.4727
Training Loss: 14.5127
Training Loss: 14.5595
Training Loss: 14.6736
Training Loss: 14.7456
Training Loss: 14.8002
Training Loss: 14.9878
Training Loss: 15.0969
Training Loss: 15.2668
Training Loss: 15.3622
Training Loss: 15.3971
Training Loss: 15.4787
Training Loss: 15.5541
Training Loss: 15.6312
Training Loss: 15.6749
Training Loss: 15.7281
Training Loss: 15.8155
Training Loss: 15.9108
Training Loss: 15.9798
Training Loss: 16.0772
Training Loss: 16.1882
Training Loss: 16.3481
Training Loss: 16.4518
Training Loss: 16.5602
Training Loss: 16.6125
Training Loss: 16.7075
Training Loss: 16.7983
Training Loss: 16.8711
Training Loss: 17.0022
Training Loss: 17.0722
Training Loss: 17.1406
Training Loss: 17.2111
Training Loss: 17.3402
Training Loss: 17.4699
Training Loss: 17.6074
Training Loss: 17.6996
Training Loss: 17.8003
Training Lo

In [ ]:
test_dataset = RetinopathyDatasetTest(csv_file = "/kaggle/input/test.csv", transform = transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

In [ ]:
with torch.no_grad():
    test_preds = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds1[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

sample = pd.read_csv("../input/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)
        